In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import scipy.stats as ss
import tensorly as tl
from jax import grad
import math
import numpy as np
from scipy.stats import norm
from scipy.special._ufuncs import gammainc
from scipy.special._ufuncs import gamma
from scipy.special._ufuncs import iv
from scipy.stats import gaussian_kde
from scipy.optimize import newton
from scipy.optimize import minimize
from scipy.stats import ncx2
from references.SABR_Free import sabrMC

try:
    tl.set_backend('jax')
except:
    print('')
tl.set_backend('jax')

# import matplotlib.pyplot as plt
# %matplotlib inline
# from IPython.display import display
# import sympy; sympy.init_printing()

# def display_matrix(m):
#     display(sympy.Matrix(m))

/Users/carolinepei/Desktop/mfe/project/citi/term_2_citi_project/citi/lib/python3.8/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


In [62]:
sample=''
sample = np.genfromtxt('references/sample.csv', delimiter=',')

In [34]:
# check distribution
print(sample.shape) # paths, time
# print(S[:,-1][:5])

# print(np.mean(sample.T[:,-1]))
# print(np.std(sample.T[:,-1]))

print(np.mean(sample.T[:,-1]) * 1.05 - 1.05)
print(np.std(sample.T[:,-1]) * (1.05**2) - 0.26667)

# 1.05
# 0.2667; =sqrt(1.05^2(e^(0.25^2*1)-1))

(100, 10000)
0.001832487937137861
0.011829146522039147


In [68]:
def put_price(r=0.05, paths = 10000, steps=100, K=1.0, maturity = 1.0, data=None):

    if data is not None:
        S=data * (1+r)
        print(type(S))
    else:
        np.random.seed(1)
        S = sabrMC(F0 = 1+r, N=paths, T=steps, n_years=1).T
        
    dt = maturity/(steps-1)         # time interval
    df = jnp.exp(-r * dt)           # discount factor per time interval
    
    H = jnp.maximum(K - S, 0)           # intrinsic values for put option
    H_np = np.
    V_np = np.zeros(H.shape)               # value matrix
    V_np[:,-1] = [i for i in H[:,-1]]

    # Valuation by LS Method
    for t in range(steps-2, 0, -1):

        good_paths = np.array(H[:,t] > 0)        # paths where the intrinsic value is positive 
                                       # the regression is performed only on these paths 

        V = jnp.array(V_np)
        rg = jnp.polyfit(S[good_paths, t], V[good_paths, t+1] * df, 2)    # polynomial regression
        C = jnp.polyval(rg, S[good_paths,t] )                             # evaluation of regression  

        exercise = np.zeros(len(good_paths), dtype=bool)           # initialize
        exercise[good_paths] = np.array(H[good_paths,t] > C)  # paths where it is optimal to exercise

        V_np[exercise,t] = np.array(H[exercise,t])              # set V equal to H where it is optimal to exercise 
        V_np[exercise,t+1:] = 0                                 # set future cash flows, for that path, equal to zero  
        discount_path = (V_np[:,t] == 0)                        # paths where we didn't exercise 
        V_np[discount_path,t] = V_np[discount_path,t+1] * df       # set V[t] in continuation region

    V0 = jnp.mean(V[:,1]) * df  # discounted expectation of V[t=1]
    return V0

print("Example price= ", put_price(data=sample.T)) # should be 0.0797

<class 'numpy.ndarray'>
Example price=  0.07373775566973997


In [69]:
print(grad(put_price, argnums=0)(0.05, paths = 10000, steps=100, K=1.0, maturity = 1.0, data=sample.T))

<class 'jax.interpreters.ad.JVPTracer'>


TracerArrayConversionError: The numpy.ndarray conversion method __array__() was called on the JAX Tracer object Traced<ConcreteArray(0.0, dtype=float64)>with<JVPTrace(level=2/0)> with
  primal = DeviceArray(0., dtype=float64)
  tangent = Traced<ShapedArray(float64[])>with<JaxprTrace(level=1/0)> with
    pval = (ShapedArray(float64[]), *)
    recipe = JaxprEqnRecipe(eqn_id=<object object at 0x136fe9990>, in_tracers=(Traced<ShapedArray(float64[1]):JaxprTrace(level=1/0)>,), out_tracer_refs=[<weakref at 0x2b2b6b590; to 'JaxprTracer' at 0x2b2b6bae0>], out_avals=[ShapedArray(float64[])], primitive=squeeze, params={'dimensions': (0,)}, effects=set(), source_info=SourceInfo(traceback=<jaxlib.xla_extension.Traceback object at 0x2b5aa01b0>, name_stack=NameStack(stack=(Transform(name='jvp'),))))
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerArrayConversionError